In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request
from vgg11 import VGG11
from imageNet_class import image_net_class_name

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"현재 디바이스: {device}")
if device.type == "cuda":
    print(f"사용 중인 GPU: {torch.cuda.get_device_name(0)}")

현재 디바이스: cuda
사용 중인 GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [9]:
def load_model(model_path):
    model = VGG11(num_classes=1000)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model


model_path = "./vgg11-8a719046.pth"
model = load_model(model_path)
print(model)


def preprocess_image(image):
    transforms = transform.Compose(
        [
            transform.Resize((224, 224)),
            transform.ToTensor(),
            transform.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    image = transforms(image).unsqueeze(0)
    return image


app = Flask(__name__)


@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error ": " No Image uploaded"}), 400

    image = request.files["image"]
    img = Image.open(image)
    img = preprocess_image(img)

    with torch.no_grad():
        outputs = model(img)
        _, pred = torch.max(outputs.data, 1)

        class_number = int(pred.item())
        print(f"class number : {class_number}")

        class_name = image_net_class_name[class_number]
        print(f"class name : {class_name}")

        prediction = str(class_name)
        test = jsonify({"prediction": prediction})
        print("json test", test)

    return jsonify({"prediction": prediction}), 200


if __name__ == "__main__":
    app.run(debug=True)


## vgg11.py, vgg11-8a719046.pth, imageNet_class, api_test 파일 동시작업

c:\ProgramData\anaconda3\envs\en03\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\envs\en03\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG11(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\ProgramData\anaconda3\envs\en03\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
